In [24]:
import pandas as pd
import numpy as np
import random
from collections import defaultdict
from datetime import timedelta
import math 
import copy

In [25]:

# Cargar archivo Excel
file_path = "generic_input_case.xlsx"

# Leer cada hoja
df_horizonte = pd.read_excel(file_path, sheet_name="HORIZONTE")
df_bd_up = pd.read_excel(file_path, sheet_name="BD_UP")
df_frota = pd.read_excel(file_path, sheet_name="FROTA")
df_grua = pd.read_excel(file_path, sheet_name="GRUA")
df_fabrica = pd.read_excel(file_path, sheet_name="FABRICA")
df_rota = pd.read_excel(file_path, sheet_name="ROTA")



c:\Users\wilma\anaconda3\Lib\site-packages\openpyxl\worksheet\_read_only.py:81: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():


In [26]:

# --- Funciones de Validación de Restricciones ---

def validate_daily_demand(schedule_df, df_fabrica,flag):
    """Verifica si el volumen diario entregado está dentro de los límites."""
    daily_volume = schedule_df.groupby('DIA')['VOLUME'].sum()
    merged = pd.merge(daily_volume.reset_index(), df_fabrica[['DIA', 'DEMANDA_MIN', 'DEMANDA_MAX']], on='DIA')

    violations = merged[
        (merged['VOLUME'] < merged['DEMANDA_MIN']) |
        (merged['VOLUME'] > merged['DEMANDA_MAX'])
    ]

    if not violations.empty:
        if flag==1:
            print("Incumplimiento - Demanda Diaria:")
            print(violations)
        return False
    return True

def validate_daily_rsp(schedule_df, df_fabrica,flag):
    """Verifica si el RSP promedio ponderado diario está dentro de los límites."""
    if schedule_df.empty or 'VOLUME' not in schedule_df.columns or 'RSP' not in schedule_df.columns:
        return True # No hay datos para validar

    schedule_df['RSP_WEIGHTED_SUM'] = schedule_df['RSP'] * schedule_df['VOLUME']
    daily_rsp_sum = schedule_df.groupby('DIA')['RSP_WEIGHTED_SUM'].sum()
    daily_volume_sum = schedule_df.groupby('DIA')['VOLUME'].sum()

    # Evitar división por cero si un día no tiene volumen
    daily_avg_rsp = (daily_rsp_sum / daily_volume_sum).fillna(0).reset_index()
    daily_avg_rsp.rename(columns={0: 'RSP_AVG'}, inplace=True)

    merged = pd.merge(daily_avg_rsp, df_fabrica[['DIA', 'RSP_MIN', 'RSP_MAX']], on='DIA')

    # Añadir tolerancia pequeña para comparaciones de punto flotante
    tolerance = 1e-6
    violations = merged[
        (round(merged['RSP_AVG'],2) < round(merged['RSP_MIN'],2)) |
        (round(merged['RSP_AVG'],2) > round(merged['RSP_MAX'],2))
    ]
     # Limpiar columna auxiliar
    # schedule_df.drop(columns=['RSP_WEIGHTED_SUM'], inplace=True, errors='ignore')


    if not violations.empty:
        if flag==1:
            print("Incumplimiento - RSP Diario:")
            print(violations)
        return False
    return True

def validate_transporter_fleet_size(schedule_df, df_frota):
    """Verifica si la flota diaria por transportador está dentro de los límites."""
    # Asegúrate de contar vehículos únicos por transportador y día
    daily_fleet = schedule_df.groupby(['DIA', 'TRANSPORTADOR'])['QTD_VEICULOS'].sum().reset_index() # Suma todos los vehiculos asignados por transportador en el día. OJO: esto asume que QTD_VEICULOS es el total por asignación UP-Transp-Dia

    # Necesitamos comparar contra FROTA_MIN y FROTA_MAX por transportador
    # df_frota no tiene DIA, se aplica a todos los días
    merged = pd.merge(daily_fleet, df_frota[['TRANSPORTADOR', 'FROTA_MIN', 'FROTA_MAX']], on='TRANSPORTADOR')

    violations = merged[
        (merged['QTD_VEICULOS'] < merged['FROTA_MIN']) |
        (merged['QTD_VEICULOS'] > merged['FROTA_MAX'])
    ]

    if not violations.empty:
        print("Incumplimiento - Tamaño Flota Transportador:")
        print(violations)
        return False
    return True

def validate_crane_capacity(schedule_df, df_grua):
    """Verifica si un transportador atiende más UPs simultáneas que sus grúas."""
    # Contar UPs únicas atendidas por cada transportador cada día
    daily_up_count = schedule_df.groupby(['DIA', 'TRANSPORTADOR'])['UP'].nunique().reset_index()
    daily_up_count.rename(columns={'UP': 'UP_COUNT'}, inplace=True)

    merged = pd.merge(daily_up_count, df_grua[['TRANSPORTADOR', 'QTD_GRUAS']], on='TRANSPORTADOR')

    violations = merged[merged['UP_COUNT'] > merged['QTD_GRUAS']]

    if not violations.empty:
        print("Incumplimiento - Capacidad Grúas:")
        print(violations)
        return False
    return True

def validate_fazenda_exclusivity(schedule_df):
    """Verifica si un transportador opera en más de una Fazenda por día."""
    daily_fazenda_count = schedule_df.groupby(['DIA', 'TRANSPORTADOR'])['FAZENDA'].nunique().reset_index()
    daily_fazenda_count.rename(columns={'FAZENDA': 'FAZENDA_COUNT'}, inplace=True)

    violations = daily_fazenda_count[daily_fazenda_count['FAZENDA_COUNT'] > 1]

    if not violations.empty:
        print("Incumplimiento - Exclusividad Fazenda:")
        print(violations)
        return False
    return True

def validate_min_vehicles_per_up(schedule_df, df_grua):
    """Verifica el porcentaje mínimo de vehículos asignados a una UP."""
    # Calcular flota total por transportador por día
    total_daily_fleet = schedule_df.groupby(['DIA', 'TRANSPORTADOR'])['QTD_VEICULOS'].sum().reset_index()
    total_daily_fleet.rename(columns={'QTD_VEICULOS': 'TOTAL_VEHICULOS_DIA'}, inplace=True)

    # Unir con la asignación por UP y la info de grúa
    merged = pd.merge(schedule_df, total_daily_fleet, on=['DIA', 'TRANSPORTADOR'])
    merged = pd.merge(merged, df_grua[['TRANSPORTADOR', 'PORCENTAGEM_VEICULOS_MIN']], on='TRANSPORTADOR')

    # Calcular porcentaje real
    merged['PORCENTAJE_REAL'] = merged['QTD_VEICULOS'] / merged['TOTAL_VEHICULOS_DIA']

    # Añadir tolerancia pequeña
    tolerance = 1e-6
    violations = merged[merged['PORCENTAJE_REAL'] < merged['PORCENTAGEM_VEICULOS_MIN'] - tolerance]

    if not violations.empty:
        print("Incumplimiento - Vehículos Mínimos por UP:")
        # Mostrar solo columnas relevantes para la violación
        print(violations[['DIA', 'TRANSPORTADOR', 'UP', 'QTD_VEICULOS', 'TOTAL_VEHICULOS_DIA', 'PORCENTAGEM_VEICULOS_MIN', 'PORCENTAJE_REAL']])
        return False
    return True

# --- Funciones de Validación de Completitud (Más Complejas) ---
# Estas requieren analizar el historial a través de los días

def check_continuity(days_list):
    """Función auxiliar para verificar continuidad y número de bloques."""
    if not days_list:
        return 0, True # 0 bloques, continuo
    
    sorted_days = sorted(list(set(days_list)))
    if not sorted_days:
      return 0, True

    blocks = 0
    is_continuous = True
    if sorted_days:
        blocks = 1
        for i in range(len(sorted_days) - 1):
            if sorted_days[i+1] != sorted_days[i] + 1:
                blocks += 1
                is_continuous = False # Hay un salto
    return blocks, is_continuous

def validate_completion_rules(schedule_df, df_bd_up):
    """Valida las reglas de transporte completo/fraccionado para UPs y Fazendas."""
    all_valid = True
    total_volume_transported = schedule_df.groupby('UP')['VOLUME'].sum()
    
    # 1. Validación por UP
    for up_id, group in schedule_df.groupby('UP'):
        up_info = df_bd_up[df_bd_up['UP'] == up_id].iloc[0]
        total_volume_up = up_info['VOLUME']
        volume_transported_up = total_volume_transported.get(up_id, 0)
        days_worked = group['DIA'].tolist()
        num_blocks, is_continuous = check_continuity(days_worked)

        is_small_up = total_volume_up < 7000

        # Regla UP Pequeña (<7000 m³)
        if is_small_up:
            # Debe ser transportada completamente si se empezó
            if 0 < volume_transported_up < total_volume_up:
                 print(f"Incumplimiento UP < 7000: UP {up_id} iniciada pero no completada. Transportado: {volume_transported_up:.2f}, Total: {total_volume_up:.2f}")
                 all_valid = False
            # Debe tener solo 1 entrada (ser continua)
            if num_blocks > 1:
                 print(f"Incumplimiento UP < 7000: UP {up_id} tiene {num_blocks} entradas (debe ser 1). Días: {sorted(list(set(days_worked)))}")
                 all_valid = False

        # Regla UP Grande (>=7000 m³)
        else:
            # Puede tener hasta 2 entradas
            if num_blocks > 2:
                 print(f"Incumplimiento UP >= 7000: UP {up_id} tiene {num_blocks} entradas (máximo 2). Días: {sorted(list(set(days_worked)))}")
                 all_valid = False

    # # # 2. Validación por Fazenda (Simplificada: verifica si fue completada si se inició)
    # # # Una validación completa requeriría verificar si el transportador cambió *antes* de completar.
    # # # Esta versión solo chequea si el volumen total fue transportado al final del horizonte.
    # fazenda_volume_transported = schedule_df.groupby('FAZENDA')['VOLUME'].sum()
    # # Calcular volumen total por Fazenda desde BD_UP
    # fazenda_total_volume = df_bd_up.groupby('FAZENDA')['VOLUME'].sum()

    # for fazenda, transported in fazenda_volume_transported.items():
    #     total = fazenda_total_volume.get(fazenda, 0)
    #     # Si se transportó algo pero no todo, es una violación potencial (simplificada)
    #     # La regla estricta es más compleja de verificar sin seguir estados diarios.
    #     if 0 < transported < total:
    #         # Esto es una indicación, no una prueba definitiva de violación de la regla *durante* la ejecución
    #         print(f"Advertencia - Completitud Fazenda: Fazenda {fazenda} iniciada pero no completada al final del horizonte. Transportado: {transported:.2f}, Total: {total:.2f}")
    #         all_valid = False # Podrías marcarlo como inválido aquí si quieres ser estricto con el resultado final

    return all_valid





In [27]:

# --- Función Principal de Validación ---
def validate_essential_constraints(schedule_df, df_fabrica, df_frota, df_grua, df_bd_up,flag):
    # """Ejecuta todas las validaciones."""
    # print("--- Iniciando Validación de Restricciones ---")
    is_feasible = True

    # Limpiar columna auxiliar de RSP si existe de ejecuciones anteriores
    if 'RSP_WEIGHTED_SUM' in schedule_df.columns:
        schedule_df.drop(columns=['RSP_WEIGHTED_SUM'], inplace=True, errors='ignore')

    if not validate_daily_demand(schedule_df, df_fabrica,flag): is_feasible = False
    if not validate_daily_rsp(schedule_df, df_fabrica,flag): is_feasible = False
    if not validate_transporter_fleet_size(schedule_df, df_frota): is_feasible = False
    if not validate_crane_capacity(schedule_df, df_grua): is_feasible = False
    # if not validate_fazenda_exclusivity(schedule_df): is_feasible = False
    
    return is_feasible

# --- Función Principal de Validación ---
def validate_all_constraints(schedule_df, df_fabrica, df_frota, df_grua, df_bd_up,flag):
    """Ejecuta todas las validaciones."""
    # print("--- Iniciando Validación de Restricciones ---")
    is_feasible = True

    # Limpiar columna auxiliar de RSP si existe de ejecuciones anteriores
    if 'RSP_WEIGHTED_SUM' in schedule_df.columns:
        schedule_df.drop(columns=['RSP_WEIGHTED_SUM'], inplace=True, errors='ignore')

    if not validate_daily_demand(schedule_df, df_fabrica,flag): is_feasible = False
    if not validate_daily_rsp(schedule_df, df_fabrica,flag): is_feasible = False
    if not validate_transporter_fleet_size(schedule_df, df_frota): is_feasible = False
    if not validate_crane_capacity(schedule_df, df_grua): is_feasible = False
    if not validate_fazenda_exclusivity(schedule_df): is_feasible = False
    # OJO: La validación de min_vehicles_per_up puede fallar si hay días/transportadores sin asignaciones en schedule_df
    # Se necesita asegurar que schedule_df contenga todas las asignaciones relevantes.
    # if not validate_min_vehicles_per_up(schedule_df, df_grua): is_feasible = False # Puede necesitar ajustes
    if not validate_completion_rules(schedule_df, df_bd_up): is_feasible = False

    print("--- Validación Terminada ---")
    if is_feasible:
        print(">>> La solución CUMPLE con todas las restricciones verificadas.")
    else:
        print(">>> La solución NO CUMPLE con una o más restricciones.")

    return is_feasible

In [28]:

# --- Funciones Auxiliares ---
def calculate_daily_transport_volume(vehicles, caixa_carga, tempo_ciclo):
    """Calcula el volumen máximo que pueden transportar los vehículos en un día."""
    if vehicles <= 0 or caixa_carga <= 0 or tempo_ciclo <= 0:
        return 0
    return vehicles * caixa_carga * tempo_ciclo

def check_continuity(days_list):
    """Función auxiliar para verificar continuidad y número de bloques."""
    if not isinstance(days_list, list) or not days_list:
         return 0, True
    
    sorted_days = sorted(list(set(filter(None, days_list)))) # Filtra None o NaNs
    if not sorted_days:
      return 0, True

    blocks = 0
    if sorted_days:
        blocks = 1
        for i in range(len(sorted_days) - 1):
            if sorted_days[i+1] != sorted_days[i] + 1:
                blocks += 1
    is_continuous = (blocks <= 1) # Consideramos continuo si hay 0 o 1 bloque
    return blocks, is_continuous


In [29]:
# --- Heurística Constructiva ---

def constructive_heuristic(df_horizonte, df_bd_up, df_frota, df_grua, df_fabrica, df_rota,flag):
    """
    Intenta construir una solución factible día por día.
    """
    schedule = [] # Lista para guardar las filas de la solución

    # --- Inicializar Estado ---
    up_status = df_bd_up.set_index('UP').copy() # Usar UP como índice para fácil acceso
    up_status['VOLUME_RESTANTE'] = up_status['VOLUME']
    # Inicializar DIAS_TRABAJADOS como una lista vacía para cada fila
    up_status['DIAS_TRABAJADOS'] = [[] for _ in range(len(up_status))]
    up_status['BLOQUES_TRABAJO'] = 0
    up_status['ESTADO'] = 'PENDIENTE' # PENDIENTE, ACTIVA, COMPLETADA, PAUSADA (>7000m3 only)

    # Estado del transportador por día (se resetea/actualiza cada día)
    transporter_daily_status = {t: {'fazenda_actual': None, 'ups_activas_hoy': [], 'vehiculos_usados_hoy': 0}
                                for t in df_frota['TRANSPORTADOR']}

    # Guardar la última Fazenda trabajada por un transportador para continuidad
    transporter_last_fazenda = {t: None for t in df_frota['TRANSPORTADOR']}
    transportadores=list(df_frota['TRANSPORTADOR'].unique())
    random.shuffle(transportadores)
    # print(up_status)
    # print("hi i'm here")
    up_status=up_status.sample(frac=1)
    # print(up_status)
    # print("--- Iniciando Heurística Constructiva ---")

    # --- Bucle Principal por Día ---
    for _, dia_info in df_horizonte.iterrows():
        dia = dia_info['DIA']
        mes = dia_info['MES']
        # print(f"\n--- Construyendo Día {dia} ---")

        # Resetear estado diario (mantener fazenda si es necesario?)
        for t in transporter_daily_status:
            transporter_daily_status[t]['ups_activas_hoy'] = []
            transporter_daily_status[t]['vehiculos_usados_hoy'] = 0
            # ¿Debería mantenerse la fazenda_actual si no se completó? Sí.
            # transporter_daily_status[t]['fazenda_actual'] = None # Resetear o mantener?

        # Obtener demandas y límites del día

        volumen_acumulado_dia = 0

        # --- Priorización de UPs ---
        # 1. UPs Activas o Pausadas (que deben continuarse)
        # 2. UPs Pendientes
        # Podría añadirse más lógica (ej. por RSP para cumplir objetivo diario)
        ups_activas_pausadas = up_status[up_status['ESTADO'].isin(['ACTIVA', 'PAUSADA'])].index.tolist()
        ups_pendientes = up_status[up_status['ESTADO'] == 'PENDIENTE'].index.tolist()
        # Simple Prio: Activas/Pausadas primero, luego Pendientes (podría ordenarse mejor)
        ups_priorizadas = ups_activas_pausadas + ups_pendientes

        # --- Bucle por Transportador ---
        # Podríamos ordenar transportadores (ej. por capacidad) pero lo hacemos simple
        
        for transportador in transportadores:
            
            transportador_info = df_frota[df_frota['TRANSPORTADOR'] == transportador].iloc[0]
            grua_info = df_grua[df_grua['TRANSPORTADOR'] == transportador].iloc[0]
            frota_min_transp = transportador_info['FROTA_MIN']
            frota_max_transp = transportador_info['FROTA_MAX']
            qtd_gruas_transp = grua_info['QTD_GRUAS']
            
            # --- Determinar Fazenda Objetivo ---
            fazenda_objetivo = None
            # Si el transportador estaba activo en una Fazenda ayer y no la completó, DEBE continuarla.
            fazenda_previa = transporter_last_fazenda[transportador]
            if fazenda_previa:
                 ups_en_fazenda_previa = up_status[up_status['FAZENDA'] == fazenda_previa]
                 # Si *alguna* UP en esa fazenda está ACTIVA o PAUSADA, debe continuarla.
                 if any(ups_en_fazenda_previa['ESTADO'].isin(['ACTIVA', 'PAUSADA'])):
                     fazenda_objetivo = fazenda_previa
                    #  print(f"  {transportador} debe continuar Fazenda: {fazenda_objetivo}")
            
            # Si no hay obligación de continuar, buscar una nueva Fazenda (o continuar una opcionalmente)
            if not fazenda_objetivo:
                 # Estrategia simple: Elegir la primera fazenda con UPs pendientes/activas que pueda atender
                 possible_fazendas = up_status[up_status.index.isin(ups_priorizadas)]['FAZENDA'].unique()
                 if (len(possible_fazendas)>0):
                     random.shuffle(possible_fazendas)
                #  print(possible_fazendas)
                 for f in possible_fazendas:
                     # Verificar si hay ruta para *alguna* UP en esa fazenda
                     ups_in_f = up_status[(up_status['FAZENDA'] == f) & (up_status.index.isin(ups_priorizadas))].index
                     if any(not df_rota[(df_rota['ORIGEM'] == up) & (df_rota['TRANSPORTADOR'] == transportador)].empty for up in ups_in_f):
                          fazenda_objetivo = f
                        #   print(f"  {transportador} elige nueva Fazenda: {fazenda_objetivo}")
                          break # Elegir la primera encontrada

            if not fazenda_objetivo:
                print(f"  {transportador} no encontró Fazenda objetivo hoy.")
                transporter_last_fazenda[transportador] = None # No trabajó
                continue # Siguiente transportador
                
            transporter_daily_status[transportador]['fazenda_actual'] = fazenda_objetivo
            transporter_last_fazenda[transportador] = fazenda_objetivo # Actualizar la última trabajada

            # --- Seleccionar UPs dentro de la Fazenda Objetivo ---
            ups_seleccionadas_hoy = []
            # Filtrar UPs priorizadas que estén en la fazenda objetivo
            ups_candidatas_fazenda = [up for up in ups_priorizadas if up_status.loc[up, 'FAZENDA'] == fazenda_objetivo]
            # print("FLAGGGGGGG", ups_candidatas_fazenda)
            for up_id in ups_candidatas_fazenda:
                if len(ups_seleccionadas_hoy) >= qtd_gruas_transp:
                    break # Límite de grúas alcanzado

                # Verificar si hay ruta
                rota_info = df_rota[(df_rota['ORIGEM'] == up_id) & (df_rota['TRANSPORTADOR'] == transportador)]
                if rota_info.empty:
                    continue

                # Verificar Reglas de Completitud/Fragmentación ANTES de añadir
                up_data = up_status.loc[up_id]
                vol_total = up_data['VOLUME']
                vol_restante = up_data['VOLUME_RESTANTE']
                estado_actual = up_data['ESTADO']
                bloques_actuales = up_data['BLOQUES_TRABAJO']

                if vol_restante <= 0: continue # Ya completada

                puede_empezar_o_continuar = False
                if estado_actual == 'PENDIENTE':
                    # Si es pequeña (<7000), solo puede empezar si se puede terminar hoy (simplificación: difícil saber a priori)
                    # Omitimos esa check compleja por ahora y permitimos empezar.
                    # Si es grande (>=7000), puede empezar si bloques < 2
                    if vol_total < 7000:
                         puede_empezar_o_continuar = True # Asumimos que puede empezar
                    elif bloques_actuales < 2:
                         puede_empezar_o_continuar = True
                elif estado_actual == 'ACTIVA':
                     puede_empezar_o_continuar = True # Puede continuar
                elif estado_actual == 'PAUSADA':
                     # Solo puede continuar si bloques < 2
                     if bloques_actuales < 2:
                          puede_empezar_o_continuar = True

                if puede_empezar_o_continuar:
                    ups_seleccionadas_hoy.append(up_id)

            if not ups_seleccionadas_hoy:
                print(f"  {transportador} no encontró UPs viables en {fazenda_objetivo} hoy.")
                continue

            # print(f"  {transportador} en {fazenda_objetivo} -> UPs: {ups_seleccionadas_hoy}")

            # --- Asignar Vehículos y Calcular Volumen ---
            num_ups_asignadas = len(ups_seleccionadas_hoy)
            vehiculos_total_asignar = 0

            # Intentar asignar al menos el mínimo de flota, distribuido
            if num_ups_asignadas > 0:
                 vehiculos_total_asignar = frota_max_transp#random.randint(frota_min_transp,frota_max_transp)

            distr=random.random()
            veh_up=[0 for _ in range (num_ups_asignadas)]
            ass_veh=vehiculos_total_asignar
            perc=grua_info["PORCENTAGEM_VEICULOS_MIN"]

            for i in range (num_ups_asignadas):
                if  (num_ups_asignadas==1):
                    veh_up[i]=int(vehiculos_total_asignar)
                    ass_veh-=veh_up[i]   
                elif distr< perc and i==0 and ass_veh>0:
                    veh_up[i]=math.ceil(perc*vehiculos_total_asignar)
                    ass_veh-=veh_up[i]
                elif distr > perc and distr<(1-perc) and i==0 and ass_veh>0:
                    veh_up[i]=math.ceil(distr*vehiculos_total_asignar)
                    ass_veh-=veh_up[i]   
                elif (distr > perc and i==0) and ass_veh>0:
                    veh_up[i]=int(vehiculos_total_asignar)
                    ass_veh-=veh_up[i]   
                   
                else:
                    veh_up[i]=min(math.ceil((1-perc)*vehiculos_total_asignar),ass_veh)
                    ass_veh-=max(veh_up[i],0)


            # Distribución simple (podría ser proporcional al volumen restante, etc.)
            vehiculos_por_up = {}
            if num_ups_asignadas > 0:
                
                for idx,up_id in enumerate(ups_seleccionadas_hoy):
                     vehi_up = veh_up[idx]

                     
                     # Verificar si con estos vehículos se puede transportar *algo*
                     rota_info = df_rota[(df_rota['ORIGEM'] == up_id) & (df_rota['TRANSPORTADOR'] == transportador)].iloc[0]
                     caixa = rota_info['CAIXA_CARGA']
                     ciclo = rota_info['TEMPO_CICLO'] # TODO: Ajustar por ciclo lento? (PDF no especifica cómo)
                     vol_potencial_up = calculate_daily_transport_volume(vehi_up, caixa, ciclo)
                     
                     if vol_potencial_up > 0:
                         vehiculos_por_up[up_id] = vehi_up

                         
                # Recalcular UPs activas y vehículos totales usados hoy
                ups_seleccionadas_hoy = list(vehiculos_por_up.keys())
                vehiculos_usados_transp_hoy = sum(vehiculos_por_up.values())
                
                # Validar flota total usada hoy
                if not (frota_min_transp <= vehiculos_usados_transp_hoy <= frota_max_transp):
                     print(f"Error interno: Flota calculada ({vehiculos_usados_transp_hoy}) fuera de rango [{frota_min_transp}, {frota_max_transp}] para {transportador}")
                     pass

                transporter_daily_status[transportador]['vehiculos_usados_hoy'] = vehiculos_usados_transp_hoy
                transporter_daily_status[transportador]['ups_activas_hoy'] = ups_seleccionadas_hoy

            # --- Calcular Volumen Transportado y Actualizar Estado ---
            for up_id in ups_seleccionadas_hoy:
                vehi_up = vehiculos_por_up[up_id]
                up_data = up_status.loc[up_id] # Obtener datos actuales
                rota_info = df_rota[(df_rota['ORIGEM'] == up_id) & (df_rota['TRANSPORTADOR'] == transportador)].iloc[0]
                caixa = rota_info['CAIXA_CARGA']
                ciclo = rota_info['TEMPO_CICLO'] # TODO: Ajustar ciclo lento
                
                vol_max_transportable_hoy = calculate_daily_transport_volume(vehi_up, caixa, ciclo)
                vol_real_transportado = min(vol_max_transportable_hoy, up_data['VOLUME_RESTANTE'])

                if vol_real_transportado > 0:
                    # Actualizar volumen restante
                    up_status.loc[up_id, 'VOLUME_RESTANTE'] -= vol_real_transportado

                    # --- Inicialización Segura de dias_previos ---
                    try:
                        # Intenta obtener la lista actual
                        retrieved_days = up_status.loc[up_id, 'DIAS_TRABAJADOS']
                        # Verifica si es una lista válida, si no, inicializa como vacía
                        if isinstance(retrieved_days, list):
                            dias_previos = retrieved_days.copy() # Usar .copy() por seguridad
                        else:
                            dias_previos = []
                    except KeyError:
                        # Error si up_id no existe (no debería pasar aquí, pero por seguridad)
                        print(f"Error Crítico: UP {up_id} no encontrado en up_status index.")
                        dias_previos = []
                    except Exception as e:
                            # Cualquier otro error inesperado al leer
                            print(f"Error inesperado al leer DIAS_TRABAJADOS para {up_id}: {e}")
                            dias_previos = []
                    # --- Fin Inicialización Segura ---

                    # Ahora estamos seguros de que 'dias_previos' es una lista

                    nuevo_estado = 'ACTIVA'

                    # Verificar si es un nuevo bloque de trabajo
                    # Un bloque nuevo empieza si la lista estaba vacía O si el día actual no es consecutivo al último registrado
                    is_new_block = False
                    if not dias_previos: # Es el primer día que se trabaja esta UP
                        is_new_block = True
                    elif dia != dias_previos[-1] + 1: # Hubo una interrupción (día no consecutivo)
                        is_new_block = True

                    # Incrementar contador de bloques SOLO si es un nuevo bloque Y NO es el primer día en absoluto
                    if is_new_block and dias_previos:
                        # Asegúrate de que BLOQUES_TRABAJO exista y sea numérico
                            if pd.isna(up_status.loc[up_id, 'BLOQUES_TRABAJO']):
                                up_status.loc[up_id, 'BLOQUES_TRABAJO'] = 0
                            up_status.loc[up_id, 'BLOQUES_TRABAJO'] += 1

                    # Añadir día actual a la lista si no está ya (por si acaso)
                    if dia not in dias_previos:
                        dias_previos.append(dia)

                    # Reasignar la lista actualizada usando .at para seguridad
                    up_status.at[up_id, 'DIAS_TRABAJADOS'] = dias_previos

                    # Actualizar estado final de la UP para hoy
                    if up_status.loc[up_id, 'VOLUME_RESTANTE'] <= 1e-6: # Usar tolerancia para flotantes
                        nuevo_estado = 'COMPLETADA'
                    # (Podrías añadir lógica para PAUSADA si es UP grande y se interrumpe, pero la dejaremos ACTIVA por ahora)
                    up_status.loc[up_id, 'ESTADO'] = nuevo_estado

                    # Añadir la entrada al schedule (DataFrame final)
                    schedule.append({
                        'UP': up_id,
                        'FAZENDA': up_data['FAZENDA'], # Necesitas obtener up_data antes en el loop
                        'TRANSPORTADOR': transportador,
                        'DIA': dia,
                        'MES': mes,
                        'DB': up_data['DB'],      # Necesitas obtener up_data antes en el loop
                        'RSP': up_data['RSP'],     # Necesitas obtener up_data antes en el loop
                        'QTD_VEICULOS': vehi_up,
                        'VOLUME': vol_real_transportado
                    })
                    volumen_acumulado_dia += vol_real_transportado
                    # print(f"    -> {transportador} transporta {vol_real_transportado:.2f} m³ desde {up_id} ({nuevo_estado})")

                    up_status.loc[up_id, 'ESTADO'] = nuevo_estado


    # --- Fin del Horizonte ---
    final_schedule_df = pd.DataFrame(schedule)

    # print("--- Heurística Constructiva Terminada ---")
    return final_schedule_df

In [30]:
def validate_single_day(schedule_df_day, df_fabrica_day, df_frota, df_grua):
    """
    Validates essential constraints for a single day's schedule.
    Returns True if feasible, False otherwise.
    """
    if schedule_df_day is None or schedule_df_day.empty:
        if df_fabrica_day is None or df_fabrica_day.empty:
            print("  ERROR: Factory data for the day is missing.")
            return False
        if 'DEMANDA_MIN' not in df_fabrica_day.columns:
            print("  ERROR: 'DEMANDA_MIN' column missing in factory data.")
            return False
        # An empty schedule is feasible only if minimum demand is 0
        return df_fabrica_day['DEMANDA_MIN'].iloc[0] <= 1e-6 # Use tolerance

    if 'DIA' not in schedule_df_day.columns or schedule_df_day.empty:
         print("  ERROR: Schedule DataFrame is empty or missing 'DIA' column.")
         return False

    # Ensure day value is valid before using it
    try:
        day = int(schedule_df_day['DIA'].iloc[0])
    except (ValueError, IndexError):
        print("  ERROR: Invalid or missing day value in schedule_df_day.")
        return False

    if not isinstance(df_fabrica_day, pd.DataFrame):
        print(f"  ERROR: df_fabrica_day is not a DataFrame for day {day}")
        return False

    fabrica_constraints = df_fabrica_day[df_fabrica_day['DIA'] == day]
    if fabrica_constraints.empty:
         print(f"  ERROR: No factory constraints found for day {day}")
         return False

    flag = 0 # Internal flag for sub-validations
    is_feasible = True
    try:
        # Ensure validation functions are defined and callable
        if 'validate_daily_demand' not in globals() or not callable(globals()['validate_daily_demand']) or \
           'validate_daily_rsp' not in globals() or not callable(globals()['validate_daily_rsp']) or \
           'validate_transporter_fleet_size' not in globals() or not callable(globals()['validate_transporter_fleet_size']) or \
           'validate_crane_capacity' not in globals() or not callable(globals()['validate_crane_capacity']) or \
           'validate_fazenda_exclusivity' not in globals() or not callable(globals()['validate_fazenda_exclusivity']):
            print("  ERROR: One or more required validation functions are not defined.")
            return False

        if not validate_daily_demand(schedule_df_day, fabrica_constraints, flag): is_feasible = False
        if not validate_daily_rsp(schedule_df_day, fabrica_constraints, flag): is_feasible = False
        if not validate_transporter_fleet_size(schedule_df_day, df_frota): is_feasible = False
        if not validate_crane_capacity(schedule_df_day, df_grua): is_feasible = False
        if not validate_fazenda_exclusivity(schedule_df_day): is_feasible = False
        # Optional: Validate min vehicles per UP
        # if 'validate_min_vehicles_per_up' in globals() and callable(globals()['validate_min_vehicles_per_up']):
        #     try:
        #         if not validate_min_vehicles_per_up(schedule_df_day, df_grua): is_feasible = False
        #     except Exception as e_min_veh:
        #         print(f"  WARN: validate_min_vehicles_per_up failed for day {day}: {e_min_veh}")
        # else:
        #     print("  WARN: validate_min_vehicles_per_up function not defined, skipping check.")

    except Exception as e:
        print(f"  ERROR: Exception during single day validation for day {day}: {e}")
        return False

    return is_feasible

def find_feasible_daily_pattern(target_day, current_up_status, df_horizonte, df_frota, df_grua, df_fabrica, df_rota, max_attempts=100):
    """
    Tries to find a feasible schedule pattern for a single target_day.
    Returns the first feasible pattern found, or the last attempt if none are feasible.
    """
    # print(f"  Attempting to find feasible pattern for Day {target_day}...") # Reduce verbosity
    horizon_day_info = df_horizonte[df_horizonte['DIA'] == target_day]
    if horizon_day_info.empty:
        print(f"  ERROR: No horizon data found for Day {target_day}. Cannot proceed.")
        return None, False # Return None and feasibility status

    mes_val = horizon_day_info['MES'].iloc[0] if 'MES' in horizon_day_info.columns else None
    ano_val = horizon_day_info['ANO'].iloc[0] if 'ANO' in horizon_day_info.columns else None
    df_horizonte_day = pd.DataFrame([{'DIA': target_day, 'MES': mes_val, 'ANO': ano_val}])

    df_fabrica_day = df_fabrica[df_fabrica['DIA'] == target_day].copy()
    if df_fabrica_day.empty:
        print(f"  ERROR: No factory data found for Day {target_day}. Cannot proceed.")
        return None, False

    last_attempt_schedule = None # Store the last generated schedule

    for attempt in range(max_attempts):
        temp_up_status_indexed = current_up_status.copy()
        temp_up_status_columnar = temp_up_status_indexed.reset_index()
        daily_schedule_attempt = None # Initialize before try block

        try:
            # Ensure constructive_heuristic function is defined and callable
            if 'constructive_heuristic' not in globals() or not callable(globals()['constructive_heuristic']):
                 print("  ERROR: `constructive_heuristic` function not defined.")
                 return None, False # Cannot proceed

            daily_schedule_attempt = constructive_heuristic(
                df_horizonte_day, temp_up_status_columnar, df_frota, df_grua,
                df_fabrica_day, df_rota, flag=0
            )
            last_attempt_schedule = daily_schedule_attempt # Store the latest attempt

        except Exception as e:
            print(f"  ERROR: Exception in constructive_heuristic attempt {attempt+1}: {e}")
            # Continue to next attempt, maybe it works
            continue

        if daily_schedule_attempt is not None and not daily_schedule_attempt.empty:
             if 'DIA' in daily_schedule_attempt.columns:
                 daily_schedule_attempt = daily_schedule_attempt[daily_schedule_attempt['DIA'] == target_day].copy()
             else:
                 print(f"  WARN: Heuristic output missing 'DIA' column (Day {target_day}, Att {attempt+1}).")
                 daily_schedule_attempt = pd.DataFrame() # Treat as empty

        # Validate this single day's schedule
        if validate_single_day(daily_schedule_attempt, df_fabrica_day, df_frota, df_grua):
            # Return the first feasible pattern found
            if not daily_schedule_attempt.empty:
                 # print(f"    Found feasible pattern Day {target_day} Att {attempt + 1}.") # Reduce verbosity
                 return daily_schedule_attempt, True # Return pattern and True for feasible
            else:
                 # Handle feasible empty schedule case
                 if 'DEMANDA_MIN' in df_fabrica_day.columns and df_fabrica_day['DEMANDA_MIN'].iloc[0] <= 1e-6:
                      # print(f"    Found feasible EMPTY pattern Day {target_day} Att {attempt + 1}.") # Reduce verbosity
                      return daily_schedule_attempt, True # Return empty pattern and True
                 # else: continue loop if empty schedule validated but demand > 0

    # If loop finishes without finding a feasible pattern
    print(f"  WARN: Failed to find feasible pattern for Day {target_day} after {max_attempts} attempts. Using last attempt.")
    # Return the last generated schedule (even if infeasible) and False
    return last_attempt_schedule, False

def calculate_extension_days(daily_pattern, current_up_status, horizon_end_day, current_day):
    """
    Calculates how many days the daily_pattern can be repeated based on volume.
    Returns 0 if the pattern cannot be applied even once (e.g., required UP is empty).
    """
    if daily_pattern is None or daily_pattern.empty: return 0

    max_k = float('inf')
    pattern_requires_volume = False

    if 'UP' not in daily_pattern.columns or 'VOLUME' not in daily_pattern.columns:
        print("  ERROR: Daily pattern missing 'UP' or 'VOLUME' column.")
        return 0

    volume_per_up_in_pattern = daily_pattern.groupby('UP')['VOLUME'].sum()

    # Check if pattern transports any volume at all
    if volume_per_up_in_pattern.empty or volume_per_up_in_pattern.max() <= 1e-6:
         days_left_in_horizon = horizon_end_day - current_day + 1
         return min(1, days_left_in_horizon) # Extend empty/zero-volume pattern by 1 day

    for up_id, daily_vol_transported in volume_per_up_in_pattern.items():
        if daily_vol_transported <= 1e-6: continue
        pattern_requires_volume = True

        if up_id not in current_up_status.index:
             print(f"  ERROR: UP {up_id} from pattern not in current_up_status index.")
             return 0 # Cannot extend

        vol_restante = current_up_status.loc[up_id, 'VOLUME_RESTANTE']

        # Check if UP required by pattern is already depleted
        if vol_restante <= 1e-6:
             # print(f"  WARN: UP {up_id} required by pattern has 0 volume. Cannot apply.") # Reduce verbosity
             return 0 # Cannot apply pattern even once

        # Avoid division by zero (already checked daily_vol_transported)
        days_possible_for_up = math.floor(vol_restante / daily_vol_transported)
        max_k = min(max_k, days_possible_for_up)

    # If no UP required volume (e.g., pattern only moves from already full UPs? Unlikely)
    if not pattern_requires_volume:
         max_k = 1 # Should be handled by initial check, but defensively set to 1

    days_left_in_horizon = horizon_end_day - current_day + 1
    max_k = min(max_k, days_left_in_horizon)

    if max_k == float('inf'):
        return min(1, days_left_in_horizon) # Should not happen if logic above is correct
    else:
        # If max_k calculation resulted in 0, it means less than 1 day's volume is available.
        # Since we passed the vol_restante check, we can apply it once.
        final_k = max(0, int(max_k))
        if final_k == 0 and pattern_requires_volume:
             final_k = 1 # Allow running for the current day
        final_k = min(final_k, days_left_in_horizon) # Ensure not exceeding horizon
        return final_k


# --- Main Control Loop for Pattern Heuristic ---
def run_pattern_based_heuristic(df_horizonte, df_bd_up, df_frota, df_grua, df_fabrica, df_rota, max_daily_attempts=100):
    """
    Orchestrates the pattern-finding and extension heuristic.
    Attempts to generate schedule entries for all days, using best-effort for failed days.
    """
    print("--- Starting Pattern-Based Heuristic (Generating entries for ALL days) ---")
    overall_schedule_list = []
    if 'DIA' not in df_horizonte.columns: print("ERROR: 'DIA' missing in df_horizonte."); return None
    current_day = df_horizonte['DIA'].min()
    horizon_end_day = df_horizonte['DIA'].max()

    if 'UP' not in df_bd_up.columns: print("ERROR: 'UP' missing in df_bd_up."); return None
    up_status = df_bd_up.set_index('UP').copy()
    if 'VOLUME' not in up_status.columns: print("ERROR: 'VOLUME' missing in up_status."); return None

    up_status['VOLUME_RESTANTE'] = up_status['VOLUME']
    up_status['DIAS_TRABAJADOS'] = [[] for _ in range(len(up_status))]
    up_status['BLOQUES_TRABAJO'] = 0
    up_status['ESTADO'] = 'PENDIENTE'

    while current_day <= horizon_end_day:
        print(f"\nProcessing Day {current_day}...")

        # 1. Try find a feasible pattern. Also get the pattern itself (even if infeasible).
        daily_pattern, is_pattern_feasible = find_feasible_daily_pattern(
            current_day, up_status, df_horizonte, df_frota, df_grua, df_fabrica, df_rota, max_daily_attempts
        )

        # If constructive heuristic failed entirely (returned None)
        if daily_pattern is None:
             print(f"  ERROR: Constructive heuristic failed completely for Day {current_day}. Skipping day.")
             current_day += 1
             continue

        # If a pattern was generated (feasible or not)
        k = 0 # Default extension days
        if is_pattern_feasible:
             # If the pattern found is feasible, calculate extension days
             k = calculate_extension_days(daily_pattern, up_status, horizon_end_day, current_day)
             if k == 0:
                  print(f"  WARN: Feasible pattern for Day {current_day} cannot be applied (k=0, UP depleted?). Applying for 1 day only.")
                  k = 1 # Force applying the feasible pattern once
             else:
                  print(f"  Feasible pattern found Day {current_day}. Extending for {k} day(s).")
        else:
             # If the pattern found was infeasible (best effort from find_feasible_daily_pattern)
             print(f"  WARN: Using infeasible best-effort pattern for Day {current_day}. Applying for 1 day only.")
             k = 1 # Apply the infeasible pattern only for the current day

        # Ensure k doesn't push beyond horizon
        k = min(k, horizon_end_day - current_day + 1)

        # Apply the pattern (feasible or infeasible) for k days and update state
        for i in range(k):
            day_to_apply = current_day + i
            if day_to_apply > horizon_end_day: break

            # Ensure daily_pattern is a DataFrame before copying
            if not isinstance(daily_pattern, pd.DataFrame):
                print(f"ERROR: daily_pattern for day {current_day} is not a DataFrame. Skipping application.")
                # This case should ideally not happen if find_feasible_daily_pattern returns correctly
                break # Break from the inner loop for this pattern

            pattern_copy = daily_pattern.copy()
            # Ensure pattern_copy is not empty before proceeding
            if pattern_copy.empty:
                 print(f"  INFO: Applying empty pattern for Day {day_to_apply}.")
                 # Add an empty df placeholder? Or just skip adding? Let's skip adding for now.
                 # overall_schedule_list.append(pattern_copy) # Avoid adding empty dfs
            else:
                 pattern_copy['DIA'] = day_to_apply
                 day_horizon_info = df_horizonte[df_horizonte['DIA'] == day_to_apply]
                 if not day_horizon_info.empty:
                      pattern_copy['MES'] = day_horizon_info['MES'].iloc[0] if 'MES' in day_horizon_info.columns else None
                 overall_schedule_list.append(pattern_copy) # Add the schedule entries

                 # Update state only if the pattern wasn't empty
                 if 'UP' not in pattern_copy.columns or 'VOLUME' not in pattern_copy.columns:
                      print(f"ERROR: Pattern copy day {day_to_apply} missing UP/VOLUME.")
                      continue

                 volume_transported_today = pattern_copy.groupby('UP')['VOLUME'].sum()
                 for up_id, vol_transported in volume_transported_today.items():
                     if up_id not in up_status.index: continue
                     if vol_transported > 1e-6:
                         try:
                             up_status.loc[up_id, 'VOLUME_RESTANTE'] -= vol_transported
                             if up_status.loc[up_id, 'VOLUME_RESTANTE'] < 0: up_status.loc[up_id, 'VOLUME_RESTANTE'] = 0
                             dias_list = up_status.at[up_id, 'DIAS_TRABAJADOS']
                             if not isinstance(dias_list, list): dias_list = []
                             last_day_worked = dias_list[-1] if dias_list else None
                             is_new_block = False
                             if last_day_worked is None: is_new_block = True
                             elif isinstance(last_day_worked, (int, float)) and day_to_apply != last_day_worked + 1:
                                 is_new_block = True
                                 if pd.isna(up_status.at[up_id, 'BLOQUES_TRABAJO']): up_status.at[up_id, 'BLOQUES_TRABAJO'] = 0
                                 if last_day_worked is not None:
                                     up_status.at[up_id, 'BLOQUES_TRABAJO'] += 1
                             if day_to_apply not in dias_list:
                                  dias_list.append(day_to_apply)
                                  up_status.at[up_id, 'DIAS_TRABAJADOS'] = dias_list
                             if up_status.loc[up_id, 'VOLUME_RESTANTE'] <= 1e-6: up_status.loc[up_id, 'ESTADO'] = 'COMPLETADA'
                             else: up_status.loc[up_id, 'ESTADO'] = 'ACTIVA'
                         except KeyError: print(f"ERROR: KeyError updating state {up_id}."); continue
                         except Exception as e: print(f"ERROR: Update state {up_id} day {day_to_apply}: {e}"); continue

        # Advance the current day by the number of days the pattern was applied
        # Ensure k is at least 1 if we applied something
        current_day += max(1, k) # Advance by at least 1 day

    print("\n--- Pattern-Based Heuristic Finished (Attempted all days) ---")
    if not overall_schedule_list: return pd.DataFrame()
    try: final_schedule = pd.concat(overall_schedule_list, ignore_index=True)
    except Exception as e: print(f"ERROR: Concat final schedule: {e}"); return None

    print("\nValidating the complete generated schedule...")
    try:
        # Ensure validate_all_constraints is defined
        # Pass flag=0 to suppress validation prints unless needed
        is_final_feasible = validate_essential_constraints(final_schedule, df_fabrica, df_frota, df_grua, df_bd_up, flag=0)
        if is_final_feasible: print(">>> Final Schedule is Feasible <<<")
        else: print(">>> FINAL SCHEDULE IS INFEASIBLE (As expected potentially, due to forced daily entries) <<<")
    except NameError: print("WARNING: `validate_all_constraints` not defined.")
    except Exception as e: print(f"ERROR: Final validation: {e}")
    return final_schedule

In [31]:
flag=0
# Run the pattern-based approach
df_horizonte1=df_horizonte#.iloc[0:15]
df_fabrica1=df_fabrica#.iloc[0:15]

# 1. Run the pattern-based heuristic to get an initial FEASIBLE solution
initial_feasible_solution = run_pattern_based_heuristic(df_horizonte, df_bd_up, df_frota, df_grua, df_fabrica, df_rota, max_daily_attempts=200)
print(initial_feasible_solution)
initial_feasible_solution.to_csv("pattern_heuristic_partial_solution.csv", index=False)


--- Starting Pattern-Based Heuristic (Generating entries for ALL days) ---

Processing Day 1...
  Feasible pattern found Day 1. Extending for 4 day(s).

Processing Day 5...
  Feasible pattern found Day 5. Extending for 2 day(s).

Processing Day 7...
  Feasible pattern found Day 7. Extending for 1 day(s).

Processing Day 8...
  Feasible pattern found Day 8. Extending for 1 day(s).

Processing Day 9...
  WARN: Feasible pattern for Day 9 cannot be applied (k=0, UP depleted?). Applying for 1 day only.

Processing Day 10...
  WARN: Feasible pattern for Day 10 cannot be applied (k=0, UP depleted?). Applying for 1 day only.

Processing Day 11...
  WARN: Feasible pattern for Day 11 cannot be applied (k=0, UP depleted?). Applying for 1 day only.

Processing Day 12...
  WARN: Feasible pattern for Day 12 cannot be applied (k=0, UP depleted?). Applying for 1 day only.

Processing Day 13...
  WARN: Feasible pattern for Day 13 cannot be applied (k=0, UP depleted?). Applying for 1 day only.

Processi

In [32]:
# --- Objective Function ---
def calculate_objective(schedule_df):
    """
    Calculates the average of daily DB variation (DB_max - DB_min).

    Args:
        schedule_df (pd.DataFrame): A complete schedule DataFrame with 'DIA' and 'DB' columns.

    Returns:
        float: The total DB variation over the horizon, or float('inf') if schedule is empty/invalid.
    """
    if schedule_df is None or schedule_df.empty or 'DIA' not in schedule_df.columns or 'DB' not in schedule_df.columns:
        return float('inf') # Cannot calculate objective for invalid input

    total_variation = 0
    # Group by day and calculate DB range for each day
    for day, group in schedule_df.groupby('DIA'):
        if not group.empty:
            db_max = group['DB'].max()
            db_min = group['DB'].min()
            total_variation += (db_max - db_min)
        # If a day has no deliveries, its variation is 0, so no need to add anything

    return total_variation/len(schedule_df.groupby('DIA'))

In [33]:
result=calculate_objective(initial_feasible_solution)
print(result)

50.91748356239294


In [34]:
# --- Neighborhood Search (Simulated Annealing Example) ---

# --- Neighborhood Move Functions ---
def move_change_vehicles(current_schedule, df_rota, df_frota):
    """
    Attempts to change the number of vehicles for a randomly chosen task.
    Tries to maintain transporter fleet limits but doesn't guarantee overall feasibility.
    """
    if current_schedule.empty: return current_schedule.copy()

    neighbor_schedule = current_schedule.copy()
    try:
        task_index = random.choice(neighbor_schedule.index)
    except IndexError:
        print("WARN (move): Cannot choose task from empty schedule.")
        return neighbor_schedule

    task = neighbor_schedule.loc[task_index]
    required_task_cols = ['TRANSPORTADOR', 'DIA', 'UP', 'QTD_VEICULOS', 'VOLUME']
    if not all(col in task for col in required_task_cols):
        print(f"WARN (move): Task at index {task_index} missing required columns. Skipping move.")
        return neighbor_schedule

    transportador = task['TRANSPORTADOR']
    day = task['DIA']
    up_id = task['UP']

    try:
        frota_info = df_frota[df_frota['TRANSPORTADOR'] == transportador].iloc[0]
        frota_min = frota_info['FROTA_MIN']
        frota_max = frota_info['FROTA_MAX']
    except IndexError:
        print(f"WARN (move): Frota info not found for {transportador}")
        return neighbor_schedule

    if 'QTD_VEICULOS' not in neighbor_schedule.columns:
        print(f"WARN (move): QTD_VEICULOS column missing in schedule. Cannot check fleet size.")
        return neighbor_schedule

    current_total_vehicles = neighbor_schedule[
        (neighbor_schedule['DIA'] == day) &
        (neighbor_schedule['TRANSPORTADOR'] == transportador)
    ]['QTD_VEICULOS'].sum()

    change = random.choice([-1, 1])
    new_vehicle_count = task['QTD_VEICULOS'] + change
    if new_vehicle_count <= 0: new_vehicle_count = 1

    new_total_vehicles = current_total_vehicles - task['QTD_VEICULOS'] + new_vehicle_count
    if not (frota_min <= new_total_vehicles <= frota_max):
        return neighbor_schedule # Skip move

    neighbor_schedule.loc[task_index, 'QTD_VEICULOS'] = new_vehicle_count

    # --- SIMPLIFICATION for Volume Recalculation ---
    # Volume is NOT recalculated here. Feasibility check handles consequences.

    return neighbor_schedule

# Add more move functions (e.g., reassign_transporter, shift_volume) - more complex

# --- Simulated Annealing Main Function ---
def simulated_annealing(initial_schedule, df_fabrica, df_frota, df_grua, df_bd_up, df_rota,
                        initial_temp=100, cooling_rate=0.99, max_iterations=1000):
    """
    Performs Simulated Annealing to optimize the schedule based on DB variation.
    Assumes initial_schedule is FEASIBLE.
    """
    print("\n--- Starting Simulated Annealing Optimization ---")
    try:
        if not validate_essential_constraints(initial_schedule, df_fabrica, df_frota, df_grua, df_bd_up,flag):
            print("ERROR: Initial schedule for SA is infeasible. Aborting optimization.")
            return initial_schedule
    except NameError:
        print("ERROR: `validate_essential_constraints` not defined. Cannot verify initial schedule for SA.")
        return initial_schedule
    except Exception as e:
        print(f"ERROR: Exception during initial validation for SA: {e}")
        return initial_schedule

    current_schedule = initial_schedule.copy()
    current_objective = calculate_objective(current_schedule)
    best_schedule = current_schedule.copy()
    best_objective = current_objective
    temperature = initial_temp

    print(f"Initial Objective (DB Variation): {current_objective:.2f}")

    for i in range(max_iterations):
        if temperature < 1e-3: break

        try:
            move_function = random.choice([move_change_vehicles])
            neighbor_schedule = move_function(current_schedule, df_rota, df_frota)
        except Exception as e:
            print(f"ERROR generating neighbor in iteration {i+1}: {e}")
            continue

        try:
            is_neighbor_feasible = validate_essential_constraints(neighbor_schedule, df_fabrica, df_frota, df_grua, df_bd_up,flag)
        except NameError:
            print("ERROR: `validate_essential_constraints` not defined. Cannot run SA.")
            return best_schedule
        except Exception as e:
            print(f"ERROR validating neighbor in iteration {i+1}: {e}")
            is_neighbor_feasible = False

        if is_neighbor_feasible:
            neighbor_objective = calculate_objective(neighbor_schedule)
            delta_objective = neighbor_objective - current_objective
            accept = False
            if delta_objective < 0: accept = True
            else:
                if temperature > 1e-9:
                    try:
                        acceptance_prob = math.exp(-delta_objective / temperature)
                        if random.random() < acceptance_prob: accept = True
                    except OverflowError: accept = False
            if accept:
                current_schedule = neighbor_schedule.copy()
                current_objective = neighbor_objective
                if current_objective < best_objective:
                    best_schedule = current_schedule.copy()
                    best_objective = current_objective

        temperature *= cooling_rate
        if (i + 1) % 100 == 0:
             print(f"  SA Iteration {i+1}/{max_iterations}, Temp: {temperature:.2f}, Current Obj: {current_objective:.2f}, Best Obj: {best_objective:.2f}")

    print(f"--- Simulated Annealing Finished ---")
    print(f"Final Best Objective (DB Variation): {best_objective:.2f}")
    return best_schedule


In [35]:
flag=0
# Run the pattern-based approach
df_horizonte1=df_horizonte#.iloc[0:15]
df_fabrica1=df_fabrica#.iloc[0:15]

# 1. Run the pattern-based heuristic to get an initial FEASIBLE solution
initial_feasible_solution = run_pattern_based_heuristic(df_horizonte, df_bd_up, df_frota, df_grua, df_fabrica, df_rota, max_daily_attempts=200)

# 2. Check if the initial solution is feasible and not None/Empty
# Need to ensure validate_essential_constraints is defined
is_initial_feasible = False
if initial_feasible_solution is not None and not initial_feasible_solution.empty:
    try:
        # Ensure validate_essential_constraints is defined and callable
        if 'validate_essential_constraints' in globals() and callable(globals()['validate_essential_constraints']):
             if validate_essential_constraints(initial_feasible_solution, df_fabrica, df_frota, df_grua, df_bd_up,flag):
                  is_initial_feasible = True
        else:
             print("WARNING: `validate_essential_constraints` function not found or not callable.")

    except Exception as e:
        print(f"ERROR validating initial solution: {e}")


if is_initial_feasible:
    print("\nInitial feasible solution found by pattern heuristic. Proceeding to SA optimization.")

    # 3. Run Simulated Annealing to optimize the feasible solution
    optimized_solution = simulated_annealing(
        initial_schedule=initial_feasible_solution,
        df_fabrica=df_fabrica,
        df_frota=df_frota,
        df_grua=df_grua,
        df_bd_up=df_bd_up, # Needed for validation inside SA
        df_rota=df_rota,   # Needed for moves like change_vehicles
        initial_temp=100,  # Tune these parameters
        cooling_rate=0.99, # Tune these parameters
        max_iterations=2000 # Tune these parameters
    )

    print("\n--- Optimized Schedule ---")
    print(optimized_solution)
    # Optional: Save the optimized solution
    # optimized_solution.to_csv("optimized_sa_solution.csv", index=False)

else:
    print("\n--- Pattern heuristic did not produce a feasible initial solution. Cannot run SA optimization. ---")
    if initial_feasible_solution is not None:
        print("Final (partial or infeasible) schedule from pattern heuristic:")
        print(initial_feasible_solution)
        # Optional: Save the partial/infeasible solution for analysis
        # initial_feasible_solution.to_csv("pattern_heuristic_partial_solution.csv", index=False)


--- Starting Pattern-Based Heuristic (Generating entries for ALL days) ---

Processing Day 1...
  Feasible pattern found Day 1. Extending for 1 day(s).

Processing Day 2...
  Feasible pattern found Day 2. Extending for 5 day(s).

Processing Day 7...
  Feasible pattern found Day 7. Extending for 1 day(s).

Processing Day 8...
  Feasible pattern found Day 8. Extending for 1 day(s).

Processing Day 9...
  Feasible pattern found Day 9. Extending for 1 day(s).

Processing Day 10...
  Feasible pattern found Day 10. Extending for 1 day(s).

Processing Day 11...
  WARN: Feasible pattern for Day 11 cannot be applied (k=0, UP depleted?). Applying for 1 day only.

Processing Day 12...
  WARN: Feasible pattern for Day 12 cannot be applied (k=0, UP depleted?). Applying for 1 day only.

Processing Day 13...
  WARN: Feasible pattern for Day 13 cannot be applied (k=0, UP depleted?). Applying for 1 day only.

Processing Day 14...
  WARN: Feasible pattern for Day 14 cannot be applied (k=0, UP depleted?)